<a href="https://colab.research.google.com/github/Kiwihead15/FCC-Works/blob/main/My_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code


###DataFrame of books info and look

In [ ]:
df_books.info()
df_books


###DataFrame of users info and look

In [ ]:
df_ratings.info()
df_ratings

###Plotting rows per user, and rows per isbn. It seems that most of users don´t rating the books, also most books have very few ratings.

In [ ]:
fig,axs = plt.subplots(1,2,figsize = (10,5))
axs[0].set(xlabel = 'isbn')
df_ratings.isbn.value_counts().plot(use_index = False, xticks = [], ylabel = 'rows', ax = axs[0])
axs[1].set(xlabel = 'user')
df_ratings.user.value_counts().plot(use_index = False, xticks = [], ax = axs[1])

##Let´s remove users with less than 200 ratings and books with less than a 100 ratings

###Users with more than 200 ratings

In [ ]:
List_Criteria_A = df_ratings.user.value_counts().loc[lambda x: x>=200]
List_Criteria_A

###Books with more than 100 ratings

In [ ]:
List_Criteria_B = df_ratings.isbn.value_counts().loc[lambda x: x>=100]
List_Criteria_B

## Ratings > 0

In [ ]:
List_Criteria_C = df_ratings.rating.loc[lambda x: x>0].index
List_Criteria_C

### DataFrame of books with more than 100 ratings > 0 from users with more than 200 ratings > 0 

In [ ]:
df_worthy_ratings = df_ratings.loc[((df_ratings.user.isin(List_Criteria_A.index))
                                    &(df_ratings.isbn.isin(List_Criteria_B.index))) 
                                    ].reset_index(drop = True)


df_worthy_ratings

Plotting rows per user and rows per isbn after cleaning

In [ ]:
fig,axs = plt.subplots(1,2,figsize = (10,5))
axs[0].set(xlabel = 'isbn')
df_worthy_ratings.isbn.value_counts().plot(use_index = False, xticks = [], ylabel = 'rows', ax = axs[0])
axs[1].set(xlabel = 'user')
df_worthy_ratings.user.value_counts().plot(use_index = False, xticks = [], ax = axs[1])

Join Both DataFrames in order to get the book´s titles for each rating

In [ ]:
#df_worthy_ratings_with_titles = pd.merge(df_worthy_ratings, df_books, on = 'isbn', how="inner",indicator=True)
df_worthy_ratings_with_titles = pd.merge(df_worthy_ratings, df_books, on = 'isbn', how="inner")
df_worthy_ratings_with_titles


###Removing duplicating ratings from the same user for the same book, keeping just the first. 

In [ ]:
df_books_ratings = df_worthy_ratings_with_titles.drop_duplicates(['title','user'], keep="first")
df_books_ratings

In [ ]:
# Convert to 2D matrıx
books_with_ratings_pivot = df_books_ratings.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
books_with_ratings_pivot.head()

In [ ]:
books_with_ratings_matrix = csr_matrix(books_with_ratings_pivot.values)

In [ ]:
# Train Model
#[‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]
model_knn = NearestNeighbors(algorithm='brute', metric='cosine')
model_knn.fit(books_with_ratings_matrix)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  X = books_with_ratings_pivot[books_with_ratings_pivot.index == book]
  X = X.to_numpy().reshape(1,-1)
  distances, indices = model_knn.kneighbors(X,n_neighbors=5)
  recommended_books = []
  for i in range(0,len(distances.flatten())):
    if i == 0:
      recommended_books.append(books_with_ratings_pivot.index[indices.flatten()[0]])
      recommended_books.append([])
    else:
      recommended_books[1].append([books_with_ratings_pivot.index[indices.flatten()[-i]], distances.flatten()[-i]])

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()